In [1]:
import tensorflow as tf
import tensorflow_federated as tff
from tensorflow.keras import models, layers
from tensorflow.keras.preprocessing import image
from tensorflow.keras.applications import ResNet101
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.losses import SparseCategoricalCrossentropy
import pandas as pd
import numpy as np
import os
from sklearn.model_selection import train_test_split

ModuleNotFoundError: No module named 'tensorflow_federated'

In [ ]:
# Step 1: Load the CSV file and image data
def load_data_from_csv_and_images(csv_file, image_folder, image_size=(224, 224)):
    # Load the CSV file
    df = pd.read_csv(csv_file)
    
    # Prepare image paths and labels
    image_paths = [os.path.join(image_folder, fname) for fname in df['filename']]
    labels = df['label'].values
    
    # Load and resize images
    images = []
    for img_path in image_paths:
        img = image.load_img(img_path, target_size=image_size)
        img_array = image.img_to_array(img) / 255.0  # Rescale the image
        images.append(img_array)
    
    # Convert images and labels to numpy arrays
    images = np.array(images)
    return images, labels

In [ ]:
 #Step 2: Local Model Creation Using ResNet-101
def create_model(num_classes):
   
    base_model = ResNet101(weights='imagenet', include_top=False, input_shape=(224, 224, 3))
    base_model.trainable = False  # Freeze the base model layers
    
    # Define custom classification head
    model = models.Sequential([
        base_model, 
        layers.GlobalAveragePooling2D(),
        layers.Dense(1024, activation='relu'),
        layers.Dropout(0.5),
        layers.Dense(num_classes, activation='softmax')
    ])
    
    model.compile(optimizer=Adam(), loss=SparseCategoricalCrossentropy(), metrics=['accuracy'])
    
    return model

In [ ]:
# Step 3: Train Local Model
def train_local_model(train_images, train_labels, val_images, val_labels, model, epochs=10, batch_size=32):
    history = model.fit(
        train_images, train_labels,
        validation_data=(val_images, val_labels),
        epochs=epochs,
        batch_size=batch_size
    )
    return history

In [ ]:
# Step 4: Federated Learning Setup (Using TensorFlow Federated)
def model_fn():
    num_classes = 3  # Adjust based on your number of classes
    model = create_model(num_classes)
    
    # Convert Keras model to TFF model
    keras_model = tff.learning.from_keras_model(
        model,
        input_spec=tf.TensorSpec(shape=[None, 224, 224, 3], dtype=tf.float32),
        loss_fn=tf.keras.losses.SparseCategoricalCrossentropy(),metrics=['accuracy']
    )
    return keras_model

# Federated Averaging Process
iterative_process = tff.learning.build_federated_averaging_process(model_fn)

In [ ]:
# Step 5: Prepare Data for Federated Learning
def create_federated_data(client_data, num_clients=5):
    federated_data = []
    client_data_split = np.array_split(client_data, num_clients)  # Split data into parts for clients
    
    for i in range(num_clients):
        client_images, client_labels = client_data_split[i]
        federated_data.append(
            tff.simulation.ClientData(client_images, client_labels)
        )
    return federated_data

In [ ]:
# Step 6: Perform Federated Learning for 10 Rounds
def federated_learning_round(federated_data):
    # Initialize the federated learning process
    state = iterative_process.initialize()

    for round_num in range(10):
        print(f"Round {round_num + 1}")
        state, metrics = iterative_process.next(state, federated_data)
        print(f"Metrics at round {round_num + 1}: {metrics}")
    
    # Return the trained global model
    return state.model

In [ ]:
# Step 7: Save the Final Global Model after Federated Learning
def save_model(model, model_filename='global_model.h5'):
    model.save(model_filename)
    print(f"Global model saved as {model_filename}")

In [1]:
# Step 8: Main function to load data, train local model, and simulate federated learning
def main():
    # Paths to your CSV file and image folder
    csv_file = "C:\\Users\\Acer\\Desktop\\Model\\Labels.csv"
    image_folder = "C:\\Users\\Acer\\Desktop\\Model\\photos"  # Folder containing the images
    
    # Load the data
    images, labels = load_data_from_csv_and_images(csv_file, image_folder)
    
    # Split the data into training and validation sets (80% train, 20% validation)
    train_images, val_images, train_labels, val_labels = train_test_split(images, labels, test_size=0.2, random_state=42)
    
    # Create the local model
    num_classes = len(np.unique(labels))  # Number of unique classes
    local_model = create_model(num_classes)
    
    # Train the model locally
    history = train_local_model(train_images, train_labels, val_images, val_labels, local_model, epochs=10)
    
    # Save the trained local model
    local_model.save('local_model.h5')
    
    # Prepare federated data for multiple clients (simulate real client data distribution)
    federated_data = create_federated_data((train_images, train_labels), num_clients=5)
    
    # Perform federated learning for 10 rounds
    global_model = federated_learning_round(federated_data)

    # Save the final global model after federated learning
    save_model(global_model)


# Run the main function
if __name__ == '__main__':
    main()

NameError: name 'load_data_from_csv_and_images' is not defined